In [ ]:
# statistical analysis of temperature data, QuantPy on YouTube
# heating degree days (HDD), cooling degree days (CDD) "average" Temperature = Tmax + Tmin / 2

# KANSAS CITY ?, 1889-01-01 to 12/31/1933
# KANSAS CITY DOWNTOWN AIRPORT, 1/1/1934 to 9/30/1972
# KANSAS CITY INTL AIRPORT, 10/1/1972 to 12/31/2021

# Tmax, Tmin, Precipitation

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

kansas_city = pd.read_csv("data/USW00003947.csv")

### Check for missing data

In [ ]:
### Checking for missing max and min temperatures #was 63 and 56 with 54 misaligned; restricted to KCI there are 0
max_temp = kansas_city[["Date","tmax"]]
min_temp = kansas_city[["Date","tmin"]]
print(max_temp.isnull().value_counts())
print(min_temp.isnull().value_counts())

count = 0
for mx, mn in zip(np.where(max_temp.isnull())[0], np.where(min_temp.isnull())[0]):
    if mx != mn:
        count += 1
print('\nNumber of misaligned null values equals', count)
###

### Calculate average temps and drop missing values

In [ ]:
kansas_city["Date"] = pd.to_datetime(kansas_city["Date"]) #Thanks skbrimmer!
kansas_city.set_index("Date", inplace=True)
kc_temps = kansas_city[["tmax", "tmin"]]

def avg_temp(row):
    return (row.tmax+row.tmin)/2

kc_temps["Tavg"] = kc_temps.apply(avg_temp,axis=1)
#drop na values here
kc_temps = kc_temps.dropna()
print(kc_temps)
print(kc_temps.describe())

### Flag winter and summer periods

In [ ]:
# why deep copy? does it matter if the same values are referenced, and why do we need completely new bits?
kc_temps_season = kc_temps.copy(deep=True)
kc_temps_season["month"] = kc_temps_season.index.month
mask = (kc_temps_season["month"] >= 4) & (kc_temps_season["month"] <= 9)
kc_temps_season["summer"] = np.where(mask,1,0)
kc_temps_season["winter"] = np.where(kc_temps_season["summer"] != 1,1,0)
print(kc_temps_season)

### Visually explore data

In [ ]:
kc_temps[-5000:].plot(figsize=(8,6))
plt.show()

### Distributions

In [ ]:
plt.figure(figsize=(8,6))
kc_temps.tmin.hist(bins=60, alpha=0.6, label="Tmin")
kc_temps.tmax.hist(bins=60, alpha=0.6, label="Tmax")
kc_temps["Tavg"].hist(bins=60, alpha=0.6, label="Tavg")
plt.legend()
plt.show()

### Summer and Winter

In [ ]:
plt.figure(figsize=(8,6))
kc_temps_season[kc_temps_season["winter"] == 1]["Tavg"].hist(bins=60, alpha=0.8, label="winter")
kc_temps_season[kc_temps_season["summer"] == 1]["Tavg"].hist(bins=60, alpha=0.8, label="summer")
plt.legend()
plt.show()

### Investigate temperature records

In [ ]:
# resample by month start, calculate mins and maxes for tmax, tmin and Tavg
date_list = kc_temps.index.tolist()
mth_kc_temps = pd.DataFrame(data=date_list, index=date_list).resample("MS")[0].agg([min,max])
mth_kc_temps["month"] = mth_kc_temps.index.month
def min_max_temps(row):
    stats = kc_temps[(kc_temps.index >= row["min"]) & (kc_temps.index <= row["max"])].agg([min, max])
    row["tmax_max"] = stats.loc["max", "tmax"]
    row["tmax_min"] = stats.loc["min", "tmax"]
    row["tmin_max"] = stats.loc["max", "tmin"]
    row["tmin_min"] = stats.loc["min", "tmin"]
    row["Tavg_max"] = stats.loc["max", "Tavg"]
    row["Tavg_min"] = stats.loc["min", "Tavg"]
    return row

mth_kc_temps = mth_kc_temps.apply(min_max_temps,axis=1)
mth_kc_temps

### Extremes on Record

In [ ]:
grouped_mths_kc = mth_kc_temps.groupby(mth_kc_temps.month)[["tmax_max", "tmax_min", "tmin_max", "tmin_min", "Tavg_max", "Tavg_min"]].agg([min, max])
grouped_mths_kc['months'] = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
grouped_mths_kc = grouped_mths_kc.set_index('months')
print(grouped_mths_kc[[("tmax_max", "max"),("tmin_min", "min"),("tmax_min", "min"),("tmin_max", "max")]])

In [ ]:
# Look at the max and min of the Tavg max and min
print(grouped_mths_kc[[("Tavg_max", "max"),("Tavg_max", "min"),("Tavg_min", "max"),("Tavg_min", "min")]])

### Decomposition of temperatures into seasonality and trends

In [ ]:
# Now, decomposition of time-series components
# trend - decreasin, constant or increasing?
# seasonality - periodic signal
# noise - variation in signal not accounted for by trend or seasonailty, a.k.a. "remainder"
from statsmodels.tsa.seasonal import seasonal_decompose
kc_temps.sort_index(inplace=True)
print(kc_temps)

In [ ]:
kc_temps["Tavg"].rolling(window = 365*10).mean().plot(figsize=(8,4), color="tab:red", title="Rolling mean over a 10 year window")
plt.show()

In [ ]:
kc_temps["Tavg"].rolling(window = 365*10).var().plot(figsize=(8,4), color="tab:red", title="Rolling variance over a 10 year window")
plt.show()

In [ ]:
# seasonal decomposition
decompose_result = seasonal_decompose(kc_temps['Tavg'], model='additive', period=int(365*10), extrapolate_trend='freq')
 
trend = decompose_result.trend
seasonal = decompose_result.seasonal
residual = decompose_result.resid
 
decompose_result.plot()
plt.show()

In [ ]:
# visualize 10 years
years_examine = 365*5
start_date = 3*years_examine
fig, axs = plt.subplots(3, figsize=(8,6))
fig.suptitle('Removed Trend and Seasonality')
axs[0].plot(trend[-start_date:-years_examine])
axs[1].plot(seasonal[-start_date:-years_examine])
axs[1].set_ylim([-25,25])
axs[2].plot(residual[-start_date:-years_examine])
axs[2].set_ylim([-20,20])
plt.show()

In [ ]:
# check residual distribution
residual.hist(bins=60, figsize=(8,6))